# Import

In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

# Data Load

In [5]:
train = pd.read_parquet('./data/train.parquet')
test = pd.read_parquet('./data/test.parquet')
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col = 0)

# Data Pre-Processing

In [6]:
#레이블(Delay)을 제외한 결측값이 존재하는 변수들을 학습 데이터의 최빈값으로 대체합니다
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)
print('Done.')

Done.


In [7]:
#질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.


In [8]:
#레이블이 없는 데이터들을 제거합니다
train_unlabeled = train[train['Delay'].isna()]
train_labeled = train.dropna()

In [9]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train_labeled.loc[:, 'Delay_num'] = train_labeled['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

Done.


C:\Users\samsung-user\AppData\Local\Temp\ipykernel_9628\1555283476.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_labeled.loc[:, 'Delay_num'] = train_labeled['Delay'].apply(lambda x: to_number(x, column_number))


In [20]:
X_labeled = train_labeled.drop(columns=['ID', 'Delay', 'Delay_num'])
X_unlabeled = train_unlabeled.drop(columns=['ID', 'Delay'])
y_labeled = train_labeled['Delay_num']
X_test = test.drop(columns=['ID'])

# Modeling

In [21]:
# 두 개의 뷰(view) 생성
X_view1 = X_labeled.iloc[:, :8]
X_view2 = X_labeled.iloc[:, 8:]

In [22]:
# Co-training 알고리즘 적용
n_iter = 10  # Co-training 반복 횟수
clf1 = DecisionTreeClassifier(random_state=123)
clf2 = DecisionTreeClassifier(random_state=123)

In [23]:
for i in range(n_iter):
    # 각각의 뷰에서 모델 학습
    clf1.fit(X_view1, y_labeled)
    clf2.fit(X_view2, y_labeled)
    print('fit Done')

    # 각 모델의 예측 결과를 사용하여 레이블 예측
    y_unlabeled_pred1 = clf1.predict(X_unlabeled.iloc[:, :8])
    y_unlabeled_pred2 = clf2.predict(X_unlabeled.iloc[:, 8:])
    print('predict Done')

    # 예측 결과가 일치하는 데이터를 레이블이 지정된 데이터로 추가
    idx_agree = np.where(y_unlabeled_pred1 == y_unlabeled_pred2)[0]
    print(f'{len(idx_agree)} / {len(y_unlabeled_pred1)}')
    if len(idx_agree) > 0:
        X_labeled = np.concatenate([X_labeled, X_unlabeled.iloc[idx_agree]], axis=0)
        y_labeled = np.concatenate([y_labeled, y_unlabeled_pred1[idx_agree]], axis=0)

        X_view1 = np.concatenate([X_view1, X_unlabeled.iloc[idx_agree, :8]], axis=0)
        X_view2 = np.concatenate([X_view2, X_unlabeled.iloc[idx_agree, 8:]], axis=0)

    # 레이블이 지정된 데이터셋에서 모델 성능 평가
    # clf1.fit(X_labeled.iloc[:, :8], y_labeled)
    # y_pred1 = clf1.predict(X.iloc[:, :8])
    # acc1 = accuracy_score(y, y_pred1)

    # clf2.fit(X_labeled[:, 5:], y_labeled)
    # y_pred2 = clf2.predict(X[:, 5:])
    # acc2 = accuracy_score(y, y_pred2)

    # print(f"Iteration {i+1}: View 1 Accuracy = {acc1:.2f}, View 2 Accuracy = {acc2:.2f}")

# 최종 모델 학습
# clf1.fit(X_labeled[:, :5], y_labeled)
# y_pred = clf1.predict(X[:, :5])
# acc = accuracy_score(y, y_pred)
# print(f"Final Accuracy = {acc:.2f}")

fit Done
predict Done
513897 / 744999
fit Done


c:\Users\samsung-user\miniconda3\envs\flight_delay_prediction\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
c:\Users\samsung-user\miniconda3\envs\flight_delay_prediction\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


predict Done
643142 / 744999


KeyboardInterrupt: 